<a href="https://colab.research.google.com/github/oogaini/Data-analysis-projects/blob/main/Tensorflow_Handwriting_recognition_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports




In [1]:
from numpy.random import seed
seed(888)

import tensorflow as tf
tf.random.set_seed(404)

In [38]:
import os
import numpy as np
import tensorflow as tf

from time import strftime

## Constants

In [37]:
X_TRAIN_PATH = 'drive/MyDrive/MNIST/digit_xtrain.csv'
X_TEST_PATH = 'drive/MyDrive/MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'drive/MyDrive/MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'drive/MyDrive/MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMG_WIDTH = 28
IMG_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMG_WIDTH * IMG_HEIGHT * CHANNELS

## Get the data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 224 ms, sys: 3.76 ms, total: 228 ms
Wall time: 473 ms


In [5]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [6]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 52.4 s, sys: 2.89 s, total: 55.3 s
Wall time: 56 s


## Explore the data

In [7]:
x_train_all.shape

(60000, 784)

In [8]:
x_test.shape

(10000, 784)

## Data preprocessing

In [9]:
# Re-scale
x_train_all, x_test = x_train_all/255.0, x_test/255.0

### Convert target values into one-hot encoding

In [10]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [11]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_train_all.shape

(60000, 10)

In [12]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

## Create validation dataset from training set

In [13]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [14]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [15]:
x_val.shape

(10000, 784)

In [16]:
x_train.shape

(50000, 784)

## Set up Tensorflow graph

In [17]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS])
y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES])

Instructions for updating:
non-resource variables are not supported in the long term


## Neural Network Architecture

#### Hyperparameters
how long to train our model, epochs, layers, nr of nodes per layer, learning rate

In [18]:
nr_epochs = 5
learning_rate = 0.0001

n_hidden1 = 512
n_hidden2 = 64

In [19]:
initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
w1 = tf.Variable(initial_value=initial_w1)

In [20]:
initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
b1 = tf.Variable(initial_value=initial_b1)

In [21]:
layer1_in = tf.matmul(X, w1) + b1

In [22]:
layer1_out = tf.nn.relu(layer1_in)

In [23]:
initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value=initial_w2)

initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
b2 = tf.Variable(initial_value=initial_b2)

layer2_in = tf.matmul(layer1_out, w2) + b2
layer2_out = tf.nn.relu(layer2_in)

In [24]:
initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value=initial_w3)

initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES ])
b3 = tf.Variable(initial_value=initial_b3)

layer3_in = tf.matmul(layer2_out, w3) + b3
output = tf.nn.softmax(layer3_in)

## Tensorboard Setup

In [39]:
# Folder for Tensorboard
folder_name = f'Model 1 at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
  os.makedirs(directory)
except OSError as exception:
  print(exception.strerror)
else:
  print('Successfully created directionaries!')

Successfully created directionaries!


## Loss, Optimization, Metrics

In [25]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output)

In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

#### Accuracy metric

In [27]:
correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
accuracy = tf.math.reduce_mean(tf.cast(correct_pred, tf.float32))

## Run session

In [28]:
sess = tf.Session()

In [29]:
# Initialize all the variables
init = tf.global_variables_initializer()
sess.run(init)

In [30]:
b3.eval(sess)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Batching the data

In [31]:
size_of_batch = 1000

In [32]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [34]:
def next_batch(batch_size, data, labels):

  global num_examples
  global index_in_epoch

  start = index_in_epoch
  index_in_epoch += batch_size
  
  if index_in_epoch > num_examples:
    start = 0 
    index_in_epoch = batch_size

  end = index_in_epoch

  return data[start:end], labels[start:end]

In [36]:
for epoch in range(nr_epochs):
  for i in range(nr_iterations):
    batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

    feed_dictionary = {X:batch_x, y:batch_y}

    sess.run(train_step, feed_dict=feed_dictionary)

    batch_accuracy = sess.run(fetches=[accuracy], feed_dict=feed_dictionary)
  print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
print('Done training!')  

Epoch 0 	| Training Accuracy = [0.376]
Epoch 1 	| Training Accuracy = [0.713]
Epoch 2 	| Training Accuracy = [0.8]
Epoch 3 	| Training Accuracy = [0.822]
Epoch 4 	| Training Accuracy = [0.839]
Done training!
